# 📝 Lezione 27 — Unsupervised Feature Engineering

---

## 🎯 Obiettivi di Apprendimento

| Obiettivo | Descrizione |
|-----------|-------------|
| **Comprendere** | Il ruolo del Feature Engineering nell'Unsupervised Learning |
| **Applicare** | Tecniche di trasformazione e creazione feature |
| **Combinare** | PCA, clustering, e altre tecniche per features migliori |
| **Valutare** | L'impatto delle nuove features sulla qualità dei modelli |

---

## 📚 Concetti Chiave

> **Feature Engineering Unsupervised**: Creare nuove feature usando tecniche non supervisionate, senza bisogno di etichette.

### 🔗 Prerequisiti
- Lezione 24: PCA (riduzione dimensionalità)
- Lezione 25: PCA + Clustering
- Lezione 26: Anomaly Detection

### 🛠️ Cosa Costruiremo
1. Features basate su PCA (componenti, varianza spiegata)
2. Features basate su clustering (distanze, appartenenza)
3. Features basate su anomaly detection (scores)
4. Feature Selection automatica

---

# 📖 1. Teoria

## 1.1 Cos'è il Feature Engineering Unsupervised?

### Il Problema
Nel Machine Learning, le **feature originali** raramente sono nella forma ottimale per i modelli.

```
Feature Engineering = Arte di trasformare i dati grezzi in feature informative
```

### Supervisionato vs Unsupervised

| Aspetto | Supervisionato | Unsupervised |
|---------|----------------|--------------|
| **Obiettivo** | Predire target | Scoprire struttura |
| **Guida** | Label y | Nessuna label |
| **Validazione** | Error vs target | Metriche interne |
| **Rischio** | Overfitting su y | Pattern spurii |

### Perché Unsupervised per Feature Engineering?

1. **Nessuna label necessaria** → Utilizzabile su dati non etichettati
2. **Scoperta automatica** → Il modello trova pattern nascosti
3. **Feature-dense** → Una componente PCA cattura info da molte feature
4. **Preprocessing universale** → Utile sia per classificazione che regressione

### Tipologie di Features Derivate

| Fonte | Tipo di Feature | Esempio |
|-------|-----------------|---------|
| PCA | Componenti | PC1, PC2, varianza cumulata |
| Clustering | Appartenenza | Cluster label, distanza da centroide |
| Anomaly | Score | Isolation score, LOF score |
| Autoencoders | Encoding | Rappresentazione latente |

---

## 1.2 Features da PCA

### Concetto Base
PCA proietta i dati in un nuovo spazio dove le **componenti catturano massima varianza**.

### Features Derivabili

| Feature | Formula | Uso |
|---------|---------|-----|
| PC_i | X @ V[:, i] | Nuove feature decorrelate |
| Varianza cumulata | sum(λ_1:k) / sum(λ) | Qualità della riduzione |
| Reconstruction Error | ||X - X_reconstructed|| | Anomaly detection |
| Loadings | V[j, i] | Importanza feature j per PC_i |

### Strategia: Componenti Come Features

```
Dati originali: 100 feature correlate
↓ PCA
Nuove feature: 10 componenti decorrelate
↓ Modello
Migliore generalizzazione
```

### Vantaggi
1. **Decorrelazione** → Migliora modelli sensibili a multicollinearità
2. **Rumore ridotto** → Ultime PC catturano rumore
3. **Interpretabilità** → Poche componenti spiegano molto
4. **Velocità** → Meno feature = training più veloce

---

## 1.3 Features da Clustering

### Concetto Base
Il clustering raggruppa dati simili. Possiamo usare questa informazione per creare feature.

### Features Derivabili

| Feature | Descrizione | Calcolo |
|---------|-------------|---------|
| **Cluster Label** | Appartenenza al cluster | kmeans.predict(X) |
| **Distance to Centroid** | Distanza dal centro del proprio cluster | dist(x, centroid_i) |
| **Distance to All** | Distanza da tutti i centroidi | [dist(x, c_0), dist(x, c_1), ...] |
| **Soft Assignment** | Probabilità di appartenenza | GaussianMixture.predict_proba() |
| **Nearest Neighbor** | Distanza dal vicino più vicino | NearestNeighbors |

### Perché Funziona?

```
Dati clienti → K-Means con k=5
↓
Cluster 0: "Alto spenditori"
Cluster 1: "Occasionali"
...
↓
Nuova feature categorica: customer_segment
```

### Use Cases

1. **Segmentazione clienti** → Feature per propensity models
2. **Anomaly context** → Anomalo rispetto al proprio cluster
3. **Multi-grain analysis** → Cluster a diversi livelli k
4. **Semi-supervised** → Propagare poche label al cluster intero

---

## 1.4 Features da Anomaly Detection

### Concetto Base
Gli algoritmi di anomaly detection producono **score** che indicano quanto un'osservazione è anomala.

### Features Derivabili

| Algoritmo | Feature | Interpretazione |
|-----------|---------|-----------------|
| Isolation Forest | `decision_function()` | Score isolamento (basso = anomalo) |
| LOF | `negative_outlier_factor_` | LOF score (basso = anomalo) |
| One-Class SVM | `decision_function()` | Distanza dal boundary |
| Autoencoder | Reconstruction Error | Errore ricostruzione |

### Uso Come Feature

```python
# Creare feature da anomaly score
from sklearn.ensemble import IsolationForest

iso = IsolationForest(contamination=0.05)
iso.fit(X)

# Nuova feature: anomaly_score
X['anomaly_score'] = iso.decision_function(X)
X['is_anomaly'] = (iso.predict(X) == -1).astype(int)
```

### Perché è Utile?

1. **Outlier-aware models** → Il modello "sa" quali punti sono strani
2. **Feature importance** → Anomaly score correlato con errori
3. **Data quality** → Identificare dati problematici
4. **Ensemble diversity** → Combinare con altre feature

---

## 1.5 Feature Selection Unsupervised

### Il Problema
Troppe feature ≠ migliori risultati. Alcune feature sono **ridondanti** o **rumorose**.

### Metodi di Selezione Unsupervised

| Metodo | Criterio | Pro | Contro |
|--------|----------|-----|--------|
| **Variance Threshold** | Rimuove bassa varianza | Semplice | Ignora correlazioni |
| **Correlation Filter** | Rimuove altamente correlate | Riduce ridondanza | Soglia arbitraria |
| **PCA Loadings** | Seleziona per importanza in PC | Data-driven | Meno interpretabile |
| **Clustering Features** | Rappresentante per cluster | Riduce gruppi simili | Scelta rappresentante |

### Variance Threshold

```python
from sklearn.feature_selection import VarianceThreshold

# Rimuove feature con varianza < 0.1
selector = VarianceThreshold(threshold=0.1)
X_selected = selector.fit_transform(X)
```

### Correlation Filter

```python
# Matrice correlazione
corr_matrix = X.corr().abs()

# Upper triangle
upper = corr_matrix.where(np.triu(np.ones(corr_matrix.shape), k=1).astype(bool))

# Colonne da rimuovere (correlazione > 0.95)
to_drop = [col for col in upper.columns if any(upper[col] > 0.95)]
```

### Schema Mentale

```
1. Rimuovi costanti (var=0)
2. Rimuovi quasi-costanti (var<threshold)
3. Rimuovi altamente correlate (>0.95)
4. (Opzionale) Seleziona top-k per loadings PCA
```

---

## 🧠 Schema Mentale — Workflow Feature Engineering Unsupervised

```
                    DATI GREZZI
                         │
        ┌────────────────┼────────────────┐
        ▼                ▼                ▼
   Preprocessing    Feature Selection  Missing/Outliers
   (Scaling)        (VarianceThreshold)  (handling)
        │                │                │
        └────────────────┼────────────────┘
                         │
                         ▼
              ┌──────────────────────┐
              │  FEATURE GENERATION  │
              └──────────────────────┘
                         │
    ┌────────────────────┼────────────────────┐
    ▼                    ▼                    ▼
  PCA               Clustering            Anomaly
  - Components      - Labels              - Scores
  - Loadings        - Distances           - Binary flags
  - Rec. Error      - Probabilities
    │                    │                    │
    └────────────────────┼────────────────────┘
                         │
                         ▼
              ┌──────────────────────┐
              │   FEATURE FUSION     │
              │  (Concatenazione)    │
              └──────────────────────┘
                         │
                         ▼
              ┌──────────────────────┐
              │  POST-SELECTION      │
              │  (Correlation Filter)│
              └──────────────────────┘
                         │
                         ▼
               FEATURES FINALI
```

### ✅ Checklist

- [ ] Dati scalati (StandardScaler)?
- [ ] Feature a bassa varianza rimosse?
- [ ] Componenti PCA aggiunte?
- [ ] Feature cluster aggiunte?
- [ ] Anomaly scores aggiunti?
- [ ] Correlazioni elevate rimosse?

In [ ]:
# =============================================================================
# 💻 DEMO 1: Features da PCA — Componenti e Loadings
# =============================================================================
"""
OBIETTIVO: Creare nuove feature usando PCA
- Componenti principali come features
- Analisi dei loadings per interpretabilità
- Reconstruction error come feature
"""

import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
from sklearn.preprocessing import StandardScaler
from sklearn.decomposition import PCA
from sklearn.datasets import make_classification

# Configurazione
np.random.seed(42)
plt.style.use('seaborn-v0_8-whitegrid')

# ═══════════════════════════════════════════════════════════════════════════
# 1. CREAZIONE DATASET CON MOLTE FEATURES
# ═══════════════════════════════════════════════════════════════════════════
print("=" * 60)
print("📊 CREAZIONE DATASET CON MOLTE FEATURES")
print("=" * 60)

# Dataset con 20 feature, ma solo alcune informative
X, y = make_classification(
    n_samples=500,
    n_features=20,
    n_informative=5,
    n_redundant=10,
    n_clusters_per_class=2,
    random_state=42
)

feature_names = [f'feature_{i}' for i in range(20)]
df_original = pd.DataFrame(X, columns=feature_names)

print(f"\n📐 Shape originale: {df_original.shape}")
print(f"📋 Features: {list(df_original.columns)[:5]}... (20 totali)")

# ═══════════════════════════════════════════════════════════════════════════
# 2. SCALING E PCA
# ═══════════════════════════════════════════════════════════════════════════
print("\n" + "=" * 60)
print("🔄 SCALING E PCA")
print("=" * 60)

# Scaling
scaler = StandardScaler()
X_scaled = scaler.fit_transform(df_original)

# PCA completa per analisi
pca_full = PCA()
pca_full.fit(X_scaled)

# Varianza spiegata
print("\n📈 Varianza spiegata per componente:")
for i, (var, cum_var) in enumerate(zip(
    pca_full.explained_variance_ratio_[:10],
    np.cumsum(pca_full.explained_variance_ratio_)[:10]
)):
    print(f"  PC{i+1}: {var:.3f} (cumulata: {cum_var:.3f})")

# ═══════════════════════════════════════════════════════════════════════════
# 3. FEATURE ENGINEERING: COMPONENTI PCA
# ═══════════════════════════════════════════════════════════════════════════
print("\n" + "=" * 60)
print("🔧 CREAZIONE FEATURES DA PCA")
print("=" * 60)

# Seleziono 5 componenti (spiegano ~80% varianza)
n_components = 5
pca = PCA(n_components=n_components)
X_pca = pca.fit_transform(X_scaled)

# Nuove feature: componenti
pca_features = pd.DataFrame(
    X_pca,
    columns=[f'PC_{i+1}' for i in range(n_components)]
)

print(f"\n✅ Create {n_components} features PCA:")
print(pca_features.head())

# Varianza spiegata totale
print(f"\n📊 Varianza spiegata totale: {pca.explained_variance_ratio_.sum():.1%}")

# ═══════════════════════════════════════════════════════════════════════════
# 4. FEATURE ENGINEERING: LOADINGS (IMPORTANZA)
# ═══════════════════════════════════════════════════════════════════════════
print("\n" + "=" * 60)
print("📊 ANALISI LOADINGS")
print("=" * 60)

# Loadings matrix
loadings = pd.DataFrame(
    pca.components_.T,
    columns=[f'PC_{i+1}' for i in range(n_components)],
    index=feature_names
)

# Top 5 feature per PC1
print("\n🔝 Top 5 feature per importanza in PC1:")
top_features_pc1 = loadings['PC_1'].abs().sort_values(ascending=False)
print(top_features_pc1.head())

# ═══════════════════════════════════════════════════════════════════════════
# 5. FEATURE ENGINEERING: RECONSTRUCTION ERROR
# ═══════════════════════════════════════════════════════════════════════════
print("\n" + "=" * 60)
print("🔄 RECONSTRUCTION ERROR COME FEATURE")
print("=" * 60)

# Ricostruzione
X_reconstructed = pca.inverse_transform(X_pca)

# Errore per ogni sample
reconstruction_error = np.sqrt(np.sum((X_scaled - X_reconstructed) ** 2, axis=1))

print(f"\n📊 Reconstruction Error Statistics:")
print(f"  Min: {reconstruction_error.min():.3f}")
print(f"  Max: {reconstruction_error.max():.3f}")
print(f"  Mean: {reconstruction_error.mean():.3f}")
print(f"  Std: {reconstruction_error.std():.3f}")

# ═══════════════════════════════════════════════════════════════════════════
# 6. DATASET FINALE CON NUOVE FEATURES
# ═══════════════════════════════════════════════════════════════════════════
print("\n" + "=" * 60)
print("📦 DATASET FINALE ARRICCHITO")
print("=" * 60)

# Combino tutto
df_enriched = pca_features.copy()
df_enriched['reconstruction_error'] = reconstruction_error

print(f"\n📐 Shape originale: {df_original.shape}")
print(f"📐 Shape arricchito: {df_enriched.shape}")
print(f"\n📋 Nuove features: {list(df_enriched.columns)}")

# ═══════════════════════════════════════════════════════════════════════════
# VISUALIZZAZIONE
# ═══════════════════════════════════════════════════════════════════════════
fig, axes = plt.subplots(1, 3, figsize=(15, 4))

# Plot 1: Varianza spiegata
ax1 = axes[0]
ax1.bar(range(1, 11), pca_full.explained_variance_ratio_[:10], alpha=0.7, label='Individuale')
ax1.plot(range(1, 11), np.cumsum(pca_full.explained_variance_ratio_)[:10], 
         'ro-', label='Cumulata')
ax1.axhline(y=0.8, color='g', linestyle='--', label='80%')
ax1.set_xlabel('Componente')
ax1.set_ylabel('Varianza Spiegata')
ax1.set_title('📊 Varianza Spiegata per Componente')
ax1.legend()

# Plot 2: Loadings PC1
ax2 = axes[1]
colors = ['green' if x > 0 else 'red' for x in loadings['PC_1']]
ax2.barh(range(len(loadings)), loadings['PC_1'], color=colors, alpha=0.7)
ax2.set_yticks(range(len(loadings)))
ax2.set_yticklabels(loadings.index, fontsize=8)
ax2.set_xlabel('Loading')
ax2.set_title('📈 Loadings della Prima Componente (PC1)')

# Plot 3: Reconstruction Error Distribution
ax3 = axes[2]
ax3.hist(reconstruction_error, bins=30, alpha=0.7, color='blue', edgecolor='black')
ax3.axvline(x=reconstruction_error.mean(), color='red', linestyle='--', 
            label=f'Media: {reconstruction_error.mean():.2f}')
ax3.set_xlabel('Reconstruction Error')
ax3.set_ylabel('Frequenza')
ax3.set_title('🔄 Distribuzione Reconstruction Error')
ax3.legend()

plt.tight_layout()
plt.show()

print("\n" + "=" * 60)
print("✅ DEMO 1 COMPLETATA!")
print("=" * 60)
print("""
📝 RIEPILOGO - Features create da PCA:
1. PC_1, PC_2, ... PC_5 → Componenti principali (decorrelate)
2. reconstruction_error → Quanto bene PCA ricostruisce il punto

💡 USO: Sostituisci le 20 feature originali con 6 features derivate!
""")

---

### 💻 Demo 2: Features da Clustering — Labels e Distanze

In [ ]:
# =============================================================================
# 💻 DEMO 2: Features da Clustering — Labels e Distanze
# =============================================================================
"""
OBIETTIVO: Creare nuove feature usando K-Means
- Cluster labels come feature categorica
- Distanze dai centroidi
- Probabilità di appartenenza (soft clustering)
"""

from sklearn.cluster import KMeans
from sklearn.mixture import GaussianMixture
from scipy.spatial.distance import cdist

# ═══════════════════════════════════════════════════════════════════════════
# 1. DATASET E CLUSTERING
# ═══════════════════════════════════════════════════════════════════════════
print("=" * 60)
print("📊 FEATURES DA CLUSTERING")
print("=" * 60)

# Uso lo stesso dataset scalato dalla demo precedente
# X_scaled già disponibile

# K-Means con 4 cluster
n_clusters = 4
kmeans = KMeans(n_clusters=n_clusters, random_state=42, n_init=10)
cluster_labels = kmeans.fit_predict(X_scaled)

print(f"\n📊 K-Means con k={n_clusters}")
print(f"📋 Distribuzione cluster:")
unique, counts = np.unique(cluster_labels, return_counts=True)
for c, cnt in zip(unique, counts):
    print(f"  Cluster {c}: {cnt} samples ({cnt/len(cluster_labels)*100:.1f}%)")

# ═══════════════════════════════════════════════════════════════════════════
# 2. FEATURE: CLUSTER LABEL (One-Hot Encoding)
# ═══════════════════════════════════════════════════════════════════════════
print("\n" + "=" * 60)
print("🏷️ FEATURE 1: CLUSTER LABEL (ONE-HOT)")
print("=" * 60)

# One-hot encoding del cluster
cluster_onehot = pd.get_dummies(cluster_labels, prefix='cluster')
print("\n📋 One-Hot Encoding:")
print(cluster_onehot.head(10))

# ═══════════════════════════════════════════════════════════════════════════
# 3. FEATURE: DISTANZA DA OGNI CENTROIDE
# ═══════════════════════════════════════════════════════════════════════════
print("\n" + "=" * 60)
print("📏 FEATURE 2: DISTANZA DAI CENTROIDI")
print("=" * 60)

# Calcolo distanze da tutti i centroidi
centroids = kmeans.cluster_centers_
distances_to_centroids = cdist(X_scaled, centroids, metric='euclidean')

dist_features = pd.DataFrame(
    distances_to_centroids,
    columns=[f'dist_to_cluster_{i}' for i in range(n_clusters)]
)

print("\n📋 Distanze dai centroidi:")
print(dist_features.head())

# ═══════════════════════════════════════════════════════════════════════════
# 4. FEATURE: DISTANZA DAL PROPRIO CENTROIDE
# ═══════════════════════════════════════════════════════════════════════════
print("\n" + "=" * 60)
print("📍 FEATURE 3: DISTANZA DAL PROPRIO CENTROIDE")
print("=" * 60)

# Distanza dal centroide del proprio cluster
own_centroid_distance = np.array([
    distances_to_centroids[i, cluster_labels[i]] 
    for i in range(len(cluster_labels))
])

print(f"\n📊 Statistiche distanza dal proprio centroide:")
print(f"  Min: {own_centroid_distance.min():.3f}")
print(f"  Max: {own_centroid_distance.max():.3f}")
print(f"  Mean: {own_centroid_distance.mean():.3f}")

# ═══════════════════════════════════════════════════════════════════════════
# 5. FEATURE: PROBABILITÀ SOFT (GAUSSIAN MIXTURE)
# ═══════════════════════════════════════════════════════════════════════════
print("\n" + "=" * 60)
print("📊 FEATURE 4: SOFT CLUSTERING (GMM)")
print("=" * 60)

# Gaussian Mixture per probabilità
gmm = GaussianMixture(n_components=n_clusters, random_state=42)
gmm.fit(X_scaled)

# Probabilità di appartenenza
probs = gmm.predict_proba(X_scaled)
prob_features = pd.DataFrame(
    probs,
    columns=[f'prob_cluster_{i}' for i in range(n_clusters)]
)

print("\n📋 Probabilità di appartenenza (GMM):")
print(prob_features.head())
print(f"\n✅ Verifica: somma righe = {prob_features.sum(axis=1).mean():.2f} (dovrebbe essere 1.0)")

# ═══════════════════════════════════════════════════════════════════════════
# 6. MULTI-GRAIN CLUSTERING
# ═══════════════════════════════════════════════════════════════════════════
print("\n" + "=" * 60)
print("🔍 FEATURE 5: MULTI-GRAIN CLUSTERING")
print("=" * 60)

# Cluster a diversi livelli di granularità
multi_grain = {}
for k in [2, 4, 8]:
    km = KMeans(n_clusters=k, random_state=42, n_init=10)
    multi_grain[f'cluster_k{k}'] = km.fit_predict(X_scaled)
    
multi_grain_df = pd.DataFrame(multi_grain)
print("\n📋 Clustering a diverse granularità:")
print(multi_grain_df.head(10))

# ═══════════════════════════════════════════════════════════════════════════
# 7. DATASET FINALE CON FEATURES CLUSTERING
# ═══════════════════════════════════════════════════════════════════════════
print("\n" + "=" * 60)
print("📦 DATASET ARRICCHITO CON FEATURES CLUSTERING")
print("=" * 60)

# Combino features più utili
df_cluster_features = pd.DataFrame({
    'cluster_label': cluster_labels,
    'dist_to_own_centroid': own_centroid_distance,
    'dist_to_cluster_0': distances_to_centroids[:, 0],
    'dist_to_cluster_1': distances_to_centroids[:, 1],
    'dist_to_cluster_2': distances_to_centroids[:, 2],
    'dist_to_cluster_3': distances_to_centroids[:, 3],
    'prob_cluster_0': probs[:, 0],
    'prob_cluster_1': probs[:, 1],
    'prob_cluster_2': probs[:, 2],
    'prob_cluster_3': probs[:, 3],
    'cluster_k2': multi_grain['cluster_k{}'.format(2)],
    'cluster_k8': multi_grain['cluster_k{}'.format(8)]
})

print(f"\n📐 Shape features clustering: {df_cluster_features.shape}")
print(f"\n📋 Colonne: {list(df_cluster_features.columns)}")

# ═══════════════════════════════════════════════════════════════════════════
# VISUALIZZAZIONE
# ═══════════════════════════════════════════════════════════════════════════
fig, axes = plt.subplots(1, 3, figsize=(15, 4))

# Plot 1: Cluster in 2D (prime 2 PC)
ax1 = axes[0]
scatter = ax1.scatter(X_pca[:, 0], X_pca[:, 1], c=cluster_labels, 
                       cmap='viridis', alpha=0.7, s=30)
# Centroidi proiettati
centroids_pca = pca.transform(centroids)
ax1.scatter(centroids_pca[:, 0], centroids_pca[:, 1], 
            c='red', marker='X', s=200, edgecolors='black', linewidths=2)
ax1.set_xlabel('PC1')
ax1.set_ylabel('PC2')
ax1.set_title('🎯 Cluster in Spazio PCA')
plt.colorbar(scatter, ax=ax1, label='Cluster')

# Plot 2: Distanza dal proprio centroide
ax2 = axes[1]
for i in range(n_clusters):
    mask = cluster_labels == i
    ax2.hist(own_centroid_distance[mask], bins=20, alpha=0.5, label=f'Cluster {i}')
ax2.set_xlabel('Distanza dal Centroide')
ax2.set_ylabel('Frequenza')
ax2.set_title('📏 Distribuzione Distanze per Cluster')
ax2.legend()

# Plot 3: Probabilità GMM (primi 20 samples)
ax3 = axes[2]
prob_sample = prob_features.iloc[:20]
bottom = np.zeros(20)
for i in range(n_clusters):
    ax3.bar(range(20), prob_sample[f'prob_cluster_{i}'], bottom=bottom, 
            label=f'Cluster {i}', alpha=0.8)
    bottom += prob_sample[f'prob_cluster_{i}'].values
ax3.set_xlabel('Sample')
ax3.set_ylabel('Probabilità')
ax3.set_title('📊 Soft Assignment (primi 20 samples)')
ax3.legend()

plt.tight_layout()
plt.show()

print("\n" + "=" * 60)
print("✅ DEMO 2 COMPLETATA!")
print("=" * 60)
print("""
📝 RIEPILOGO - Features create da Clustering:
1. cluster_label → Appartenenza (categorica)
2. dist_to_own_centroid → Quanto è "tipico" nel suo cluster
3. dist_to_cluster_X → Distanze da tutti i centroidi
4. prob_cluster_X → Probabilità soft (GMM)
5. cluster_k2, k8 → Multi-grain clustering

💡 USO: Aggiunge informazione sulla struttura locale dei dati!
""")

---

### 💻 Demo 3: Features da Anomaly Detection

In [ ]:
# =============================================================================
# 💻 DEMO 3: Features da Anomaly Detection
# =============================================================================
"""
OBIETTIVO: Creare features usando Anomaly Scores
- Isolation Forest score
- LOF score
- Binary anomaly flag
- Score combination
"""

from sklearn.ensemble import IsolationForest
from sklearn.neighbors import LocalOutlierFactor

# ═══════════════════════════════════════════════════════════════════════════
# 1. ISOLATION FOREST FEATURES
# ═══════════════════════════════════════════════════════════════════════════
print("=" * 60)
print("🌲 FEATURES DA ISOLATION FOREST")
print("=" * 60)

# Isolation Forest
iso = IsolationForest(n_estimators=100, contamination=0.05, random_state=42)
iso_pred = iso.fit_predict(X_scaled)
iso_score = iso.decision_function(X_scaled)

print(f"\n📊 Isolation Forest Results:")
print(f"  Anomalie rilevate: {(iso_pred == -1).sum()} ({(iso_pred == -1).mean()*100:.1f}%)")
print(f"\n📈 Score Statistics:")
print(f"  Min: {iso_score.min():.3f}")
print(f"  Max: {iso_score.max():.3f}")
print(f"  Mean: {iso_score.mean():.3f}")

# ═══════════════════════════════════════════════════════════════════════════
# 2. LOF FEATURES
# ═══════════════════════════════════════════════════════════════════════════
print("\n" + "=" * 60)
print("🔍 FEATURES DA LOCAL OUTLIER FACTOR")
print("=" * 60)

# LOF
lof = LocalOutlierFactor(n_neighbors=20, contamination=0.05)
lof_pred = lof.fit_predict(X_scaled)
lof_score = -lof.negative_outlier_factor_  # Convertiamo in positivo per coerenza

print(f"\n📊 LOF Results:")
print(f"  Anomalie rilevate: {(lof_pred == -1).sum()} ({(lof_pred == -1).mean()*100:.1f}%)")
print(f"\n📈 Score Statistics:")
print(f"  Min: {lof_score.min():.3f}")
print(f"  Max: {lof_score.max():.3f}")
print(f"  Mean: {lof_score.mean():.3f}")

# ═══════════════════════════════════════════════════════════════════════════
# 3. NORMALIZZAZIONE SCORES
# ═══════════════════════════════════════════════════════════════════════════
print("\n" + "=" * 60)
print("📏 NORMALIZZAZIONE SCORES [0, 1]")
print("=" * 60)

def normalize_score(score):
    """Normalizza score a range [0, 1]"""
    return (score - score.min()) / (score.max() - score.min())

# Normalizzati (1 = più anomalo)
iso_score_norm = 1 - normalize_score(iso_score)  # Invertiamo perché basso = anomalo
lof_score_norm = normalize_score(lof_score)       # Alto = anomalo già OK

print(f"\n📊 Scores normalizzati:")
print(f"  ISO range: [{iso_score_norm.min():.3f}, {iso_score_norm.max():.3f}]")
print(f"  LOF range: [{lof_score_norm.min():.3f}, {lof_score_norm.max():.3f}]")

# ═══════════════════════════════════════════════════════════════════════════
# 4. FEATURES COMBINATE
# ═══════════════════════════════════════════════════════════════════════════
print("\n" + "=" * 60)
print("🔗 FEATURES COMBINATE")
print("=" * 60)

# Combinazioni
avg_anomaly_score = (iso_score_norm + lof_score_norm) / 2
max_anomaly_score = np.maximum(iso_score_norm, lof_score_norm)
consensus_anomaly = ((iso_pred == -1) & (lof_pred == -1)).astype(int)

print(f"\n📊 Consensus (entrambi dicono anomalia): {consensus_anomaly.sum()} samples")
print(f"📊 Solo ISO anomalia: {((iso_pred == -1) & (lof_pred == 1)).sum()} samples")
print(f"📊 Solo LOF anomalia: {((iso_pred == 1) & (lof_pred == -1)).sum()} samples")

# ═══════════════════════════════════════════════════════════════════════════
# 5. DATASET CON FEATURES ANOMALY
# ═══════════════════════════════════════════════════════════════════════════
print("\n" + "=" * 60)
print("📦 DATASET CON FEATURES ANOMALY")
print("=" * 60)

df_anomaly_features = pd.DataFrame({
    'iso_score': iso_score,                    # Score raw
    'iso_score_norm': iso_score_norm,          # Score normalizzato
    'iso_is_anomaly': (iso_pred == -1).astype(int),
    'lof_score': lof_score,
    'lof_score_norm': lof_score_norm,
    'lof_is_anomaly': (lof_pred == -1).astype(int),
    'avg_anomaly_score': avg_anomaly_score,    # Media scores
    'max_anomaly_score': max_anomaly_score,    # Max (più conservativo)
    'consensus_anomaly': consensus_anomaly      # Entrambi d'accordo
})

print(f"\n📐 Shape: {df_anomaly_features.shape}")
print(f"\n📋 Features create:")
for col in df_anomaly_features.columns:
    print(f"  • {col}")

# ═══════════════════════════════════════════════════════════════════════════
# VISUALIZZAZIONE
# ═══════════════════════════════════════════════════════════════════════════
fig, axes = plt.subplots(1, 3, figsize=(15, 4))

# Plot 1: Distribuzione scores
ax1 = axes[0]
ax1.hist(iso_score_norm, bins=30, alpha=0.5, label='ISO', color='blue')
ax1.hist(lof_score_norm, bins=30, alpha=0.5, label='LOF', color='orange')
ax1.axvline(x=np.percentile(iso_score_norm, 95), color='blue', linestyle='--')
ax1.axvline(x=np.percentile(lof_score_norm, 95), color='orange', linestyle='--')
ax1.set_xlabel('Anomaly Score (normalizzato)')
ax1.set_ylabel('Frequenza')
ax1.set_title('📊 Distribuzione Anomaly Scores')
ax1.legend()

# Plot 2: Scatter ISO vs LOF
ax2 = axes[1]
colors = ['red' if c == 1 else 'blue' for c in consensus_anomaly]
ax2.scatter(iso_score_norm, lof_score_norm, c=colors, alpha=0.5, s=20)
ax2.axhline(y=0.9, color='gray', linestyle='--', alpha=0.5)
ax2.axvline(x=0.9, color='gray', linestyle='--', alpha=0.5)
ax2.set_xlabel('Isolation Forest Score')
ax2.set_ylabel('LOF Score')
ax2.set_title('🔍 ISO vs LOF (rosso = consensus)')

# Plot 3: Anomalie in spazio PCA
ax3 = axes[2]
normal_mask = consensus_anomaly == 0
anomaly_mask = consensus_anomaly == 1
ax3.scatter(X_pca[normal_mask, 0], X_pca[normal_mask, 1], 
            c='blue', alpha=0.3, s=20, label='Normale')
ax3.scatter(X_pca[anomaly_mask, 0], X_pca[anomaly_mask, 1], 
            c='red', alpha=0.8, s=50, marker='X', label='Anomalia')
ax3.set_xlabel('PC1')
ax3.set_ylabel('PC2')
ax3.set_title('🎯 Anomalie in Spazio PCA')
ax3.legend()

plt.tight_layout()
plt.show()

print("\n" + "=" * 60)
print("✅ DEMO 3 COMPLETATA!")
print("=" * 60)
print("""
📝 RIEPILOGO - Features create da Anomaly Detection:
1. iso_score / lof_score → Score raw
2. iso_score_norm / lof_score_norm → Score normalizzato [0,1]
3. iso_is_anomaly / lof_is_anomaly → Flag binario
4. avg_anomaly_score → Media dei due scores
5. max_anomaly_score → Score più conservativo
6. consensus_anomaly → Entrambi d'accordo

💡 USO: Segnala al modello quali punti sono "strani"!
""")

---

### 💻 Demo 4: Feature Selection Unsupervised

In [ ]:
# =============================================================================
# 💻 DEMO 4: Feature Selection Unsupervised
# =============================================================================
"""
OBIETTIVO: Selezionare le feature più informative senza usare labels
- Variance Threshold
- Correlation Filter
- PCA-based selection
"""

from sklearn.feature_selection import VarianceThreshold

# ═══════════════════════════════════════════════════════════════════════════
# 1. DATASET CON FEATURES PROBLEMATICHE
# ═══════════════════════════════════════════════════════════════════════════
print("=" * 60)
print("📊 DATASET CON FEATURES PROBLEMATICHE")
print("=" * 60)

# Creo dataset con problemi tipici
np.random.seed(42)
n = 500

df_problem = pd.DataFrame({
    # Features buone
    'good_1': np.random.randn(n),
    'good_2': np.random.randn(n) * 2 + 5,
    'good_3': np.random.randn(n) * 0.5,
    
    # Feature costante
    'constant': np.ones(n) * 42,
    
    # Feature quasi-costante
    'low_var': np.random.choice([0, 1], size=n, p=[0.99, 0.01]),
    
    # Features altamente correlate
    'corr_1': np.random.randn(n),
})
df_problem['corr_2'] = df_problem['corr_1'] * 0.95 + np.random.randn(n) * 0.1
df_problem['corr_3'] = df_problem['corr_1'] * -0.98 + np.random.randn(n) * 0.05

print(f"\n📐 Shape iniziale: {df_problem.shape}")
print(f"\n📋 Colonne: {list(df_problem.columns)}")
print(f"\n📊 Varianze:")
for col in df_problem.columns:
    print(f"  {col}: {df_problem[col].var():.6f}")

# ═══════════════════════════════════════════════════════════════════════════
# 2. VARIANCE THRESHOLD
# ═══════════════════════════════════════════════════════════════════════════
print("\n" + "=" * 60)
print("📏 STEP 1: VARIANCE THRESHOLD")
print("=" * 60)

# Prima scalo per confrontare varianze
scaler_sel = StandardScaler()
X_scaled_sel = scaler_sel.fit_transform(df_problem)
X_scaled_sel_df = pd.DataFrame(X_scaled_sel, columns=df_problem.columns)

# Varianze dopo scaling
print("\n📊 Varianze dopo StandardScaler (dovrebbero essere ~1):")
for col in X_scaled_sel_df.columns:
    var = X_scaled_sel_df[col].var()
    status = "❌ BASSA" if var < 0.01 else "✅"
    print(f"  {col}: {var:.6f} {status}")

# Variance threshold
var_selector = VarianceThreshold(threshold=0.01)
X_var_selected = var_selector.fit_transform(X_scaled_sel_df)
selected_mask = var_selector.get_support()
selected_cols = df_problem.columns[selected_mask].tolist()

print(f"\n✂️ Rimosse features con varianza < 0.01:")
removed = [c for c, s in zip(df_problem.columns, selected_mask) if not s]
print(f"  Rimosse: {removed}")
print(f"  Rimaste: {selected_cols}")

# ═══════════════════════════════════════════════════════════════════════════
# 3. CORRELATION FILTER
# ═══════════════════════════════════════════════════════════════════════════
print("\n" + "=" * 60)
print("🔗 STEP 2: CORRELATION FILTER")
print("=" * 60)

# Lavoro solo con le feature sopravvissute
X_after_var = pd.DataFrame(X_var_selected, columns=selected_cols)

# Matrice correlazione
corr_matrix = X_after_var.corr().abs()
print("\n📊 Matrice Correlazione (valori assoluti):")
print(corr_matrix.round(3))

# Upper triangle
upper = corr_matrix.where(np.triu(np.ones(corr_matrix.shape), k=1).astype(bool))

# Trova colonne con correlazione > 0.9
threshold_corr = 0.9
to_drop_corr = []
for col in upper.columns:
    highly_corr = upper[col][upper[col] > threshold_corr].index.tolist()
    if highly_corr:
        print(f"\n⚠️ {col} altamente correlata con: {highly_corr}")
        # Rimuovi le successive (mantieni la prima)
        for hc in highly_corr:
            if hc not in to_drop_corr:
                to_drop_corr.append(hc)

print(f"\n✂️ Features da rimuovere (corr > {threshold_corr}): {to_drop_corr}")

# Applica filtro
X_after_corr = X_after_var.drop(columns=to_drop_corr)
print(f"\n✅ Features finali: {list(X_after_corr.columns)}")

# ═══════════════════════════════════════════════════════════════════════════
# 4. PIPELINE COMPLETA
# ═══════════════════════════════════════════════════════════════════════════
print("\n" + "=" * 60)
print("🔧 PIPELINE FEATURE SELECTION COMPLETA")
print("=" * 60)

def unsupervised_feature_selection(df, var_threshold=0.01, corr_threshold=0.9, verbose=True):
    """
    Pipeline completa di feature selection unsupervised
    
    Returns:
        selected_df: DataFrame con feature selezionate
        report: Dizionario con dettagli selezione
    """
    report = {'original_features': list(df.columns)}
    
    # Step 1: Scaling
    scaler = StandardScaler()
    X_scaled = pd.DataFrame(scaler.fit_transform(df), columns=df.columns)
    
    # Step 2: Variance Threshold
    var_sel = VarianceThreshold(threshold=var_threshold)
    X_var = var_sel.fit_transform(X_scaled)
    var_mask = var_sel.get_support()
    var_cols = df.columns[var_mask].tolist()
    report['after_variance'] = var_cols
    report['removed_low_var'] = [c for c, m in zip(df.columns, var_mask) if not m]
    
    if verbose:
        print(f"📏 Dopo Variance Threshold: {len(var_cols)}/{len(df.columns)} features")
    
    # Step 3: Correlation Filter
    X_var_df = pd.DataFrame(X_var, columns=var_cols)
    corr = X_var_df.corr().abs()
    upper = corr.where(np.triu(np.ones(corr.shape), k=1).astype(bool))
    
    to_drop = []
    for col in upper.columns:
        if any(upper[col] > corr_threshold):
            to_drop.append(col)
    
    final_cols = [c for c in var_cols if c not in to_drop]
    report['after_correlation'] = final_cols
    report['removed_high_corr'] = to_drop
    
    if verbose:
        print(f"🔗 Dopo Correlation Filter: {len(final_cols)}/{len(var_cols)} features")
        print(f"✅ Features finali: {final_cols}")
    
    return X_var_df[final_cols], report

# Eseguo
X_selected, report = unsupervised_feature_selection(df_problem)

print(f"\n📊 REPORT:")
print(f"  Originali: {len(report['original_features'])}")
print(f"  Dopo variance: {len(report['after_variance'])}")
print(f"  Dopo correlation: {len(report['after_correlation'])}")
print(f"  Rimosse (bassa var): {report['removed_low_var']}")
print(f"  Rimosse (alta corr): {report['removed_high_corr']}")

# ═══════════════════════════════════════════════════════════════════════════
# VISUALIZZAZIONE
# ═══════════════════════════════════════════════════════════════════════════
fig, axes = plt.subplots(1, 3, figsize=(15, 4))

# Plot 1: Varianze originali
ax1 = axes[0]
variances = df_problem.var().sort_values(ascending=False)
colors = ['red' if v < 0.01 else 'green' for v in variances]
ax1.barh(range(len(variances)), variances.values, color=colors)
ax1.set_yticks(range(len(variances)))
ax1.set_yticklabels(variances.index)
ax1.axvline(x=0.01, color='black', linestyle='--', label='Threshold')
ax1.set_xlabel('Varianza')
ax1.set_title('📊 Varianze Features (rosso = da rimuovere)')

# Plot 2: Correlation Heatmap (prima)
ax2 = axes[1]
im = ax2.imshow(X_after_var.corr(), cmap='coolwarm', aspect='auto', vmin=-1, vmax=1)
ax2.set_xticks(range(len(X_after_var.columns)))
ax2.set_yticks(range(len(X_after_var.columns)))
ax2.set_xticklabels(X_after_var.columns, rotation=45, ha='right')
ax2.set_yticklabels(X_after_var.columns)
ax2.set_title('🔗 Correlazioni (prima del filtro)')
plt.colorbar(im, ax=ax2)

# Plot 3: Features prima e dopo
ax3 = axes[2]
categories = ['Originali', 'Post-Variance', 'Post-Correlation']
counts = [len(report['original_features']), 
          len(report['after_variance']), 
          len(report['after_correlation'])]
colors = ['lightblue', 'steelblue', 'darkblue']
bars = ax3.bar(categories, counts, color=colors)
ax3.set_ylabel('Numero Features')
ax3.set_title('📉 Riduzione Features')
for bar, count in zip(bars, counts):
    ax3.text(bar.get_x() + bar.get_width()/2, bar.get_height() + 0.1, 
             str(count), ha='center', va='bottom', fontsize=12)

plt.tight_layout()
plt.show()

print("\n" + "=" * 60)
print("✅ DEMO 4 COMPLETATA!")
print("=" * 60)
print("""
📝 RIEPILOGO - Feature Selection Unsupervised:
1. Variance Threshold → Rimuove costanti e quasi-costanti
2. Correlation Filter → Rimuove ridondanti (>0.9 correlate)
3. Pipeline → Combina entrambi in un workflow

💡 USO: Pulisci il dataset PRIMA di creare nuove features!
""")

---

### 💻 Demo 5: Pipeline Completa di Feature Engineering

In [ ]:
# =============================================================================
# 💻 DEMO 5: Pipeline Completa di Feature Engineering
# =============================================================================
"""
OBIETTIVO: Combinare TUTTE le tecniche in una pipeline end-to-end
- Dataset originale → Features arricchite
- PCA + Clustering + Anomaly + Selection
"""

# ═══════════════════════════════════════════════════════════════════════════
# 1. CLASSE FEATURE ENGINEERING
# ═══════════════════════════════════════════════════════════════════════════
print("=" * 60)
print("🔧 PIPELINE COMPLETA FEATURE ENGINEERING")
print("=" * 60)

class UnsupervisedFeatureEngineer:
    """
    Pipeline completa di Feature Engineering Unsupervised.
    
    Genera features da:
    - PCA (componenti, reconstruction error)
    - K-Means (label, distanze)
    - Anomaly Detection (scores)
    """
    
    def __init__(self, n_pca_components=5, n_clusters=4, contamination=0.05):
        self.n_pca_components = n_pca_components
        self.n_clusters = n_clusters
        self.contamination = contamination
        
        # Modelli
        self.scaler = StandardScaler()
        self.pca = PCA(n_components=n_pca_components)
        self.kmeans = KMeans(n_clusters=n_clusters, random_state=42, n_init=10)
        self.iso = IsolationForest(contamination=contamination, random_state=42)
        
        self.fitted = False
        
    def fit(self, X):
        """Fit tutti i modelli"""
        # Scaling
        X_scaled = self.scaler.fit_transform(X)
        
        # Fit modelli
        self.pca.fit(X_scaled)
        self.kmeans.fit(X_scaled)
        self.iso.fit(X_scaled)
        
        self.fitted = True
        return self
    
    def transform(self, X):
        """Genera tutte le features"""
        if not self.fitted:
            raise ValueError("Chiamare fit() prima di transform()")
        
        X_scaled = self.scaler.transform(X)
        features = {}
        
        # === PCA Features ===
        X_pca = self.pca.transform(X_scaled)
        for i in range(self.n_pca_components):
            features[f'pca_{i+1}'] = X_pca[:, i]
        
        # Reconstruction error
        X_reconstructed = self.pca.inverse_transform(X_pca)
        features['pca_rec_error'] = np.sqrt(np.sum((X_scaled - X_reconstructed) ** 2, axis=1))
        
        # === Clustering Features ===
        features['cluster_label'] = self.kmeans.predict(X_scaled)
        
        # Distanza dal proprio centroide
        distances = cdist(X_scaled, self.kmeans.cluster_centers_)
        features['dist_to_centroid'] = np.array([
            distances[i, features['cluster_label'][i]] 
            for i in range(len(X_scaled))
        ])
        
        # Distanze da tutti i centroidi
        for i in range(self.n_clusters):
            features[f'dist_cluster_{i}'] = distances[:, i]
        
        # === Anomaly Features ===
        features['iso_score'] = self.iso.decision_function(X_scaled)
        features['is_anomaly'] = (self.iso.predict(X_scaled) == -1).astype(int)
        
        return pd.DataFrame(features)
    
    def fit_transform(self, X):
        """Fit e transform in un passo"""
        self.fit(X)
        return self.transform(X)
    
    def get_feature_names(self):
        """Ritorna nomi delle features generate"""
        names = []
        names += [f'pca_{i+1}' for i in range(self.n_pca_components)]
        names.append('pca_rec_error')
        names.append('cluster_label')
        names.append('dist_to_centroid')
        names += [f'dist_cluster_{i}' for i in range(self.n_clusters)]
        names += ['iso_score', 'is_anomaly']
        return names

# ═══════════════════════════════════════════════════════════════════════════
# 2. APPLICAZIONE PIPELINE
# ═══════════════════════════════════════════════════════════════════════════
print("\n📊 Dataset originale:")
print(f"  Shape: {df_original.shape}")

# Creo pipeline
engineer = UnsupervisedFeatureEngineer(
    n_pca_components=5,
    n_clusters=4,
    contamination=0.05
)

# Applico
df_engineered = engineer.fit_transform(df_original)

print(f"\n✨ Dataset arricchito:")
print(f"  Shape: {df_engineered.shape}")
print(f"\n📋 Features generate:")
for col in df_engineered.columns:
    print(f"  • {col}")

# ═══════════════════════════════════════════════════════════════════════════
# 3. COMBINO CON ORIGINALI (OPZIONALE)
# ═══════════════════════════════════════════════════════════════════════════
print("\n" + "=" * 60)
print("📦 COMBINAZIONE FEATURES")
print("=" * 60)

# Opzione 1: Solo features generate (sostituzione)
df_option1 = df_engineered.copy()
print(f"\n📐 Opzione 1 (solo generate): {df_option1.shape}")

# Opzione 2: Originali + generate (augmentation)
df_option2 = pd.concat([df_original.reset_index(drop=True), df_engineered], axis=1)
print(f"📐 Opzione 2 (originali + generate): {df_option2.shape}")

# Opzione 3: Selezione mista
df_option3 = df_engineered[['pca_1', 'pca_2', 'pca_3', 'cluster_label', 
                             'dist_to_centroid', 'iso_score']].copy()
print(f"📐 Opzione 3 (selezione): {df_option3.shape}")

# ═══════════════════════════════════════════════════════════════════════════
# 4. STATISTICHE FEATURES
# ═══════════════════════════════════════════════════════════════════════════
print("\n" + "=" * 60)
print("📊 STATISTICHE FEATURES GENERATE")
print("=" * 60)

print("\n📈 Statistiche descrittive:")
print(df_engineered.describe().round(3))

# ═══════════════════════════════════════════════════════════════════════════
# VISUALIZZAZIONE
# ═══════════════════════════════════════════════════════════════════════════
fig, axes = plt.subplots(2, 2, figsize=(12, 10))

# Plot 1: PCA space con cluster
ax1 = axes[0, 0]
scatter = ax1.scatter(df_engineered['pca_1'], df_engineered['pca_2'], 
                       c=df_engineered['cluster_label'], cmap='viridis', 
                       alpha=0.7, s=30)
ax1.set_xlabel('PCA 1')
ax1.set_ylabel('PCA 2')
ax1.set_title('🎯 Spazio PCA colorato per Cluster')
plt.colorbar(scatter, ax=ax1, label='Cluster')

# Plot 2: Distanza dal centroide per cluster
ax2 = axes[0, 1]
for c in range(4):
    mask = df_engineered['cluster_label'] == c
    ax2.hist(df_engineered.loc[mask, 'dist_to_centroid'], bins=15, 
             alpha=0.5, label=f'Cluster {c}')
ax2.set_xlabel('Distanza dal Centroide')
ax2.set_ylabel('Frequenza')
ax2.set_title('📏 Distanze per Cluster')
ax2.legend()

# Plot 3: Anomaly score vs reconstruction error
ax3 = axes[1, 0]
colors = ['red' if a == 1 else 'blue' for a in df_engineered['is_anomaly']]
ax3.scatter(df_engineered['iso_score'], df_engineered['pca_rec_error'], 
            c=colors, alpha=0.5, s=20)
ax3.set_xlabel('Isolation Score')
ax3.set_ylabel('Reconstruction Error')
ax3.set_title('🔍 ISO Score vs Rec. Error (rosso=anomalia)')

# Plot 4: Correlation heatmap features generate
ax4 = axes[1, 1]
# Seleziono solo features numeriche continue
numeric_cols = ['pca_1', 'pca_2', 'pca_3', 'pca_rec_error', 
                'dist_to_centroid', 'iso_score']
corr = df_engineered[numeric_cols].corr()
im = ax4.imshow(corr, cmap='coolwarm', aspect='auto', vmin=-1, vmax=1)
ax4.set_xticks(range(len(numeric_cols)))
ax4.set_yticks(range(len(numeric_cols)))
ax4.set_xticklabels(numeric_cols, rotation=45, ha='right', fontsize=8)
ax4.set_yticklabels(numeric_cols, fontsize=8)
ax4.set_title('🔗 Correlazione Features Generate')
plt.colorbar(im, ax=ax4)

plt.tight_layout()
plt.show()

print("\n" + "=" * 60)
print("✅ DEMO 5 COMPLETATA!")
print("=" * 60)
print("""
📝 RIEPILOGO - Pipeline Completa:

CLASSE UnsupervisedFeatureEngineer:
• fit(X) → Allena tutti i modelli
• transform(X) → Genera features
• fit_transform(X) → Entrambi

FEATURES GENERATE:
• pca_1...pca_n → Componenti PCA
• pca_rec_error → Errore ricostruzione
• cluster_label → Etichetta cluster
• dist_to_centroid → Distanza dal centro
• dist_cluster_X → Distanza da tutti i centri
• iso_score → Anomaly score
• is_anomaly → Flag binario

💡 USO: Una classe riutilizzabile per qualsiasi progetto!
""")

---

# ✍️ 3. Esercizi Pratici

## Esercizio 27.1 — Feature Engineering per Segmentazione Clienti

### 📋 Problema
Hai un dataset di comportamento clienti. Devi creare features arricchite per una successiva segmentazione.

### 🎯 Tasks
1. Carica il dataset simulato (fornito nel codice)
2. Applica PCA e estrai le prime 3 componenti
3. Esegui K-Means con k=3 ed estrai:
   - Cluster label
   - Distanza dal proprio centroide
4. Aggiungi anomaly score con Isolation Forest
5. Crea un DataFrame finale con tutte le nuove features

### 💡 Suggerimenti
- Scala sempre i dati prima di applicare PCA e clustering
- Normalizza gli anomaly scores per renderli comparabili

In [ ]:
# =============================================================================
# ✅ SOLUZIONE ESERCIZIO 27.1 — Feature Engineering Segmentazione Clienti
# =============================================================================

import numpy as np
import pandas as pd
from sklearn.preprocessing import StandardScaler
from sklearn.decomposition import PCA
from sklearn.cluster import KMeans
from sklearn.ensemble import IsolationForest
from scipy.spatial.distance import cdist
import matplotlib.pyplot as plt

np.random.seed(42)

# ═══════════════════════════════════════════════════════════════════════════
# 1. CREAZIONE DATASET CLIENTI
# ═══════════════════════════════════════════════════════════════════════════
print("=" * 60)
print("📊 ESERCIZIO 27.1 — SEGMENTAZIONE CLIENTI")
print("=" * 60)

n_customers = 400

# Simulo 3 segmenti di clienti
segment_sizes = [150, 150, 100]  # Alto, Medio, Low value

df_customers = pd.DataFrame({
    # Segment 1: High Value
    'monthly_spending': np.concatenate([
        np.random.normal(500, 100, segment_sizes[0]),  # Alto
        np.random.normal(150, 50, segment_sizes[1]),   # Medio
        np.random.normal(30, 15, segment_sizes[2])     # Basso
    ]),
    'visit_frequency': np.concatenate([
        np.random.normal(15, 3, segment_sizes[0]),
        np.random.normal(8, 2, segment_sizes[1]),
        np.random.normal(2, 1, segment_sizes[2])
    ]),
    'avg_basket_size': np.concatenate([
        np.random.normal(80, 15, segment_sizes[0]),
        np.random.normal(40, 10, segment_sizes[1]),
        np.random.normal(20, 8, segment_sizes[2])
    ]),
    'days_since_last_purchase': np.concatenate([
        np.random.exponential(5, segment_sizes[0]),    # Frequenti
        np.random.exponential(15, segment_sizes[1]),   # Occasionali
        np.random.exponential(45, segment_sizes[2])    # Rari
    ]),
    'online_engagement': np.concatenate([
        np.random.normal(70, 15, segment_sizes[0]),
        np.random.normal(45, 20, segment_sizes[1]),
        np.random.normal(15, 10, segment_sizes[2])
    ])
})

# Aggiungo alcune anomalie
anomaly_idx = [0, 50, 200, 350]
df_customers.loc[anomaly_idx, 'monthly_spending'] *= 3

print(f"\n📐 Dataset clienti: {df_customers.shape}")
print(f"\n📋 Features:")
for col in df_customers.columns:
    print(f"  • {col}: min={df_customers[col].min():.1f}, max={df_customers[col].max():.1f}")

# ═══════════════════════════════════════════════════════════════════════════
# 2. PREPROCESSING: SCALING
# ═══════════════════════════════════════════════════════════════════════════
print("\n" + "=" * 60)
print("🔄 SCALING")
print("=" * 60)

scaler = StandardScaler()
X_scaled = scaler.fit_transform(df_customers)

print("✅ Dati scalati con StandardScaler")

# ═══════════════════════════════════════════════════════════════════════════
# 3. FEATURES PCA
# ═══════════════════════════════════════════════════════════════════════════
print("\n" + "=" * 60)
print("📊 FEATURES PCA")
print("=" * 60)

pca = PCA(n_components=3)
X_pca = pca.fit_transform(X_scaled)

# Features PCA
pca_features = pd.DataFrame({
    'PC1': X_pca[:, 0],
    'PC2': X_pca[:, 1],
    'PC3': X_pca[:, 2]
})

print(f"\n📈 Varianza spiegata per componente:")
for i, var in enumerate(pca.explained_variance_ratio_):
    print(f"  PC{i+1}: {var:.1%}")
print(f"  Totale: {pca.explained_variance_ratio_.sum():.1%}")

# ═══════════════════════════════════════════════════════════════════════════
# 4. FEATURES CLUSTERING
# ═══════════════════════════════════════════════════════════════════════════
print("\n" + "=" * 60)
print("🎯 FEATURES CLUSTERING (K-MEANS k=3)")
print("=" * 60)

kmeans = KMeans(n_clusters=3, random_state=42, n_init=10)
cluster_labels = kmeans.fit_predict(X_scaled)

# Distanze
centroids = kmeans.cluster_centers_
distances = cdist(X_scaled, centroids)
dist_own = np.array([distances[i, cluster_labels[i]] for i in range(len(X_scaled))])

# Features clustering
cluster_features = pd.DataFrame({
    'customer_segment': cluster_labels,
    'dist_to_segment_center': dist_own
})

print(f"\n📊 Distribuzione segmenti:")
for c in range(3):
    count = (cluster_labels == c).sum()
    print(f"  Segmento {c}: {count} clienti ({count/len(cluster_labels)*100:.1f}%)")

# ═══════════════════════════════════════════════════════════════════════════
# 5. FEATURES ANOMALY
# ═══════════════════════════════════════════════════════════════════════════
print("\n" + "=" * 60)
print("🔍 FEATURES ANOMALY DETECTION")
print("=" * 60)

iso = IsolationForest(contamination=0.05, random_state=42)
iso_pred = iso.fit_predict(X_scaled)
iso_score = iso.decision_function(X_scaled)

# Normalizzazione score [0, 1] dove 1 = più anomalo
iso_score_norm = 1 - (iso_score - iso_score.min()) / (iso_score.max() - iso_score.min())

# Features anomaly
anomaly_features = pd.DataFrame({
    'anomaly_score': iso_score_norm,
    'is_outlier': (iso_pred == -1).astype(int)
})

print(f"\n⚠️ Anomalie rilevate: {anomaly_features['is_outlier'].sum()} clienti")

# ═══════════════════════════════════════════════════════════════════════════
# 6. DATAFRAME FINALE
# ═══════════════════════════════════════════════════════════════════════════
print("\n" + "=" * 60)
print("📦 DATASET FINALE ARRICCHITO")
print("=" * 60)

# Combino tutto
df_enriched = pd.concat([
    df_customers.reset_index(drop=True),
    pca_features,
    cluster_features,
    anomaly_features
], axis=1)

print(f"\n📐 Shape originale: {df_customers.shape}")
print(f"📐 Shape arricchito: {df_enriched.shape}")
print(f"\n📋 Nuove features aggiunte:")
new_features = list(pca_features.columns) + list(cluster_features.columns) + list(anomaly_features.columns)
for f in new_features:
    print(f"  • {f}")

# ═══════════════════════════════════════════════════════════════════════════
# VISUALIZZAZIONE
# ═══════════════════════════════════════════════════════════════════════════
fig, axes = plt.subplots(1, 3, figsize=(15, 4))

# Plot 1: Segmenti in spazio PCA
ax1 = axes[0]
scatter = ax1.scatter(df_enriched['PC1'], df_enriched['PC2'], 
                       c=df_enriched['customer_segment'], cmap='viridis', alpha=0.6)
ax1.set_xlabel('PC1')
ax1.set_ylabel('PC2')
ax1.set_title('🎯 Segmenti Clienti in Spazio PCA')
plt.colorbar(scatter, ax=ax1, label='Segmento')

# Plot 2: Anomaly score distribution
ax2 = axes[1]
for seg in range(3):
    mask = df_enriched['customer_segment'] == seg
    ax2.hist(df_enriched.loc[mask, 'anomaly_score'], bins=20, alpha=0.5, label=f'Segmento {seg}')
ax2.axvline(x=0.9, color='red', linestyle='--', label='Soglia anomalia')
ax2.set_xlabel('Anomaly Score')
ax2.set_ylabel('Frequenza')
ax2.set_title('📊 Distribuzione Anomaly Score per Segmento')
ax2.legend()

# Plot 3: Spending vs Anomaly Score
ax3 = axes[2]
colors = ['red' if o == 1 else 'blue' for o in df_enriched['is_outlier']]
ax3.scatter(df_enriched['monthly_spending'], df_enriched['anomaly_score'], 
            c=colors, alpha=0.5, s=20)
ax3.set_xlabel('Monthly Spending (€)')
ax3.set_ylabel('Anomaly Score')
ax3.set_title('💰 Spending vs Anomaly (rosso = outlier)')

plt.tight_layout()
plt.show()

# Sample output
print("\n📋 Esempio prime 5 righe arricchite:")
print(df_enriched[new_features].head().round(3))

print("\n" + "=" * 60)
print("✅ ESERCIZIO 27.1 COMPLETATO!")
print("=" * 60)

---

## Esercizio 27.2 — Feature Selection su Dataset Ridondante

### 📋 Problema
Un dataset contiene molte features ridondanti e correlate. Devi selezionare le features più informative.

### 🎯 Tasks
1. Crea un dataset con features problematiche (fornito)
2. Rimuovi features con varianza < 0.01 (dopo scaling)
3. Rimuovi features con correlazione > 0.85
4. Confronta dimensionalità prima e dopo

### 💡 Suggerimenti
- Lavora sui dati scalati per confrontare varianze
- Mantieni sempre la "prima" delle features correlate

In [ ]:
# =============================================================================
# ✅ SOLUZIONE ESERCIZIO 27.2 — Feature Selection
# =============================================================================

from sklearn.feature_selection import VarianceThreshold

np.random.seed(42)

# ═══════════════════════════════════════════════════════════════════════════
# 1. CREAZIONE DATASET PROBLEMATICO
# ═══════════════════════════════════════════════════════════════════════════
print("=" * 60)
print("📊 ESERCIZIO 27.2 — FEATURE SELECTION")
print("=" * 60)

n = 300

# Features base
base_feat = np.random.randn(n)

df_redundant = pd.DataFrame({
    # Features buone
    'feat_a': np.random.randn(n) * 2,
    'feat_b': np.random.randn(n) * 1.5 + 3,
    'feat_c': np.random.exponential(2, n),
    
    # Features costanti/quasi-costanti
    'const_1': np.ones(n) * 10,
    'quasi_const': np.random.choice([0, 1], n, p=[0.995, 0.005]),
    
    # Features altamente correlate
    'corr_base': base_feat,
    'corr_1': base_feat * 1.02 + np.random.randn(n) * 0.05,
    'corr_2': base_feat * 0.98 + np.random.randn(n) * 0.08,
    'corr_3': -base_feat * 1.01 + np.random.randn(n) * 0.03,  # Correlazione negativa
    
    # Altra feature buona
    'feat_d': np.random.randn(n) * 3 - 2
})

print(f"\n📐 Shape iniziale: {df_redundant.shape}")
print(f"\n📋 Features problematiche attese:")
print("  • const_1: costante")
print("  • quasi_const: varianza quasi zero")
print("  • corr_1, corr_2, corr_3: correlate con corr_base")

# ═══════════════════════════════════════════════════════════════════════════
# 2. SCALING
# ═══════════════════════════════════════════════════════════════════════════
print("\n" + "=" * 60)
print("🔄 STEP 1: SCALING")
print("=" * 60)

scaler = StandardScaler()
X_scaled = scaler.fit_transform(df_redundant)
df_scaled = pd.DataFrame(X_scaled, columns=df_redundant.columns)

print("\n📊 Varianze dopo scaling:")
for col in df_scaled.columns:
    var = df_scaled[col].var()
    status = "⚠️ BASSA" if var < 0.01 else "✅"
    print(f"  {col}: {var:.6f} {status}")

# ═══════════════════════════════════════════════════════════════════════════
# 3. VARIANCE THRESHOLD
# ═══════════════════════════════════════════════════════════════════════════
print("\n" + "=" * 60)
print("📏 STEP 2: VARIANCE THRESHOLD (< 0.01)")
print("=" * 60)

var_selector = VarianceThreshold(threshold=0.01)
X_after_var = var_selector.fit_transform(df_scaled)

# Features mantenute
var_mask = var_selector.get_support()
cols_after_var = df_scaled.columns[var_mask].tolist()
cols_removed_var = df_scaled.columns[~var_mask].tolist()

print(f"\n✂️ Rimosse: {cols_removed_var}")
print(f"✅ Mantenute: {cols_after_var}")
print(f"\n📐 Shape: {df_scaled.shape} → {X_after_var.shape}")

# ═══════════════════════════════════════════════════════════════════════════
# 4. CORRELATION FILTER
# ═══════════════════════════════════════════════════════════════════════════
print("\n" + "=" * 60)
print("🔗 STEP 3: CORRELATION FILTER (> 0.85)")
print("=" * 60)

df_after_var = pd.DataFrame(X_after_var, columns=cols_after_var)

# Matrice correlazione
corr_matrix = df_after_var.corr().abs()
print("\n📊 Matrice correlazione (abs):")
print(corr_matrix.round(3))

# Upper triangle
upper_tri = corr_matrix.where(
    np.triu(np.ones(corr_matrix.shape), k=1).astype(bool)
)

# Trova colonne da rimuovere
threshold = 0.85
cols_to_remove = set()

for col in upper_tri.columns:
    highly_corr = upper_tri[col][upper_tri[col] > threshold].index.tolist()
    if highly_corr:
        print(f"\n⚠️ {col} correlata con: {highly_corr} (corr > {threshold})")
        for hc in highly_corr:
            cols_to_remove.add(hc)

print(f"\n✂️ Da rimuovere: {list(cols_to_remove)}")

# Applico filtro
final_cols = [c for c in cols_after_var if c not in cols_to_remove]
df_final = df_after_var[final_cols]

print(f"✅ Features finali: {final_cols}")
print(f"\n📐 Shape: {df_after_var.shape} → {df_final.shape}")

# ═══════════════════════════════════════════════════════════════════════════
# 5. RIEPILOGO
# ═══════════════════════════════════════════════════════════════════════════
print("\n" + "=" * 60)
print("📊 RIEPILOGO SELEZIONE")
print("=" * 60)

print(f"\n📉 Riduzione features:")
print(f"  Originali: {df_redundant.shape[1]}")
print(f"  Dopo variance threshold: {len(cols_after_var)}")
print(f"  Dopo correlation filter: {len(final_cols)}")
print(f"\n📉 Riduzione totale: {df_redundant.shape[1]} → {len(final_cols)} ({(1-len(final_cols)/df_redundant.shape[1])*100:.1f}% rimosso)")

# ═══════════════════════════════════════════════════════════════════════════
# VISUALIZZAZIONE
# ═══════════════════════════════════════════════════════════════════════════
fig, axes = plt.subplots(1, 3, figsize=(15, 4))

# Plot 1: Varianze
ax1 = axes[0]
variances = df_scaled.var().sort_values()
colors = ['red' if v < 0.01 else 'green' for v in variances]
ax1.barh(range(len(variances)), variances.values, color=colors)
ax1.set_yticks(range(len(variances)))
ax1.set_yticklabels(variances.index)
ax1.axvline(x=0.01, color='black', linestyle='--', label='Threshold 0.01')
ax1.set_xlabel('Varianza')
ax1.set_title('📊 Varianze Features')
ax1.legend()

# Plot 2: Correlazione prima
ax2 = axes[1]
im = ax2.imshow(df_after_var.corr(), cmap='coolwarm', vmin=-1, vmax=1)
ax2.set_xticks(range(len(df_after_var.columns)))
ax2.set_yticks(range(len(df_after_var.columns)))
ax2.set_xticklabels(df_after_var.columns, rotation=45, ha='right', fontsize=8)
ax2.set_yticklabels(df_after_var.columns, fontsize=8)
ax2.set_title('🔗 Correlazioni (dopo variance filter)')
plt.colorbar(im, ax=ax2)

# Plot 3: Correlazione dopo
ax3 = axes[2]
im3 = ax3.imshow(df_final.corr(), cmap='coolwarm', vmin=-1, vmax=1)
ax3.set_xticks(range(len(df_final.columns)))
ax3.set_yticks(range(len(df_final.columns)))
ax3.set_xticklabels(df_final.columns, rotation=45, ha='right', fontsize=8)
ax3.set_yticklabels(df_final.columns, fontsize=8)
ax3.set_title('✅ Correlazioni FINALI')
plt.colorbar(im3, ax=ax3)

plt.tight_layout()
plt.show()

print("\n" + "=" * 60)
print("✅ ESERCIZIO 27.2 COMPLETATO!")
print("=" * 60)

---

## Esercizio 27.3 — Multi-Grain Feature Engineering

### 📋 Problema
Vuoi catturare pattern a diverse scale usando clustering a livelli multipli.

### 🎯 Tasks
1. Applica K-Means con k=2, 4, 8 (3 livelli di granularità)
2. Per ogni livello, estrai: cluster label e distanza dal centroide
3. Combina tutte le features in un unico DataFrame
4. Visualizza come i cluster si annidano tra loro

### 💡 Suggerimenti
- Usa nomi descrittivi: `cluster_k2`, `cluster_k4`, etc.
- La distanza dal centroide è utile per identificare punti "borderline"

In [ ]:
# =============================================================================
# ✅ SOLUZIONE ESERCIZIO 27.3 — Multi-Grain Feature Engineering
# =============================================================================

np.random.seed(42)

# ═══════════════════════════════════════════════════════════════════════════
# 1. DATASET
# ═══════════════════════════════════════════════════════════════════════════
print("=" * 60)
print("📊 ESERCIZIO 27.3 — MULTI-GRAIN CLUSTERING")
print("=" * 60)

# Creo dataset con struttura gerarchica
from sklearn.datasets import make_blobs

X_blobs, y_blobs = make_blobs(
    n_samples=400, 
    centers=8,  # 8 cluster "base"
    cluster_std=0.8,
    random_state=42
)

df_blobs = pd.DataFrame(X_blobs, columns=['x', 'y'])

print(f"\n📐 Dataset: {df_blobs.shape}")
print("📋 Struttura attesa: 8 cluster che si possono raggruppare in 4, poi in 2")

# Scaling
scaler = StandardScaler()
X_scaled = scaler.fit_transform(df_blobs)

# ═══════════════════════════════════════════════════════════════════════════
# 2. MULTI-GRAIN CLUSTERING
# ═══════════════════════════════════════════════════════════════════════════
print("\n" + "=" * 60)
print("🎯 CLUSTERING A 3 LIVELLI")
print("=" * 60)

k_values = [2, 4, 8]
multi_grain_features = {}

for k in k_values:
    print(f"\n📊 K-Means con k={k}:")
    
    # Fit
    km = KMeans(n_clusters=k, random_state=42, n_init=10)
    labels = km.fit_predict(X_scaled)
    
    # Distanze dai centroidi
    centroids = km.cluster_centers_
    distances = cdist(X_scaled, centroids)
    dist_own = np.array([distances[i, labels[i]] for i in range(len(X_scaled))])
    
    # Salvo features
    multi_grain_features[f'cluster_k{k}'] = labels
    multi_grain_features[f'dist_k{k}'] = dist_own
    
    # Statistiche
    for c in range(k):
        count = (labels == c).sum()
        print(f"  Cluster {c}: {count} punti ({count/len(labels)*100:.1f}%)")

# DataFrame features
df_multi_grain = pd.DataFrame(multi_grain_features)

print(f"\n📐 Features generate: {df_multi_grain.shape}")
print(f"📋 Colonne: {list(df_multi_grain.columns)}")

# ═══════════════════════════════════════════════════════════════════════════
# 3. ANALISI ANNIDAMENTO
# ═══════════════════════════════════════════════════════════════════════════
print("\n" + "=" * 60)
print("🔍 ANALISI ANNIDAMENTO CLUSTER")
print("=" * 60)

# Come si distribuiscono i cluster k=8 nei cluster k=4?
print("\n📊 Cluster k=8 → Cluster k=4:")
crosstab_8_4 = pd.crosstab(df_multi_grain['cluster_k8'], df_multi_grain['cluster_k4'])
print(crosstab_8_4)

# Come si distribuiscono i cluster k=4 nei cluster k=2?
print("\n📊 Cluster k=4 → Cluster k=2:")
crosstab_4_2 = pd.crosstab(df_multi_grain['cluster_k4'], df_multi_grain['cluster_k2'])
print(crosstab_4_2)

# ═══════════════════════════════════════════════════════════════════════════
# 4. FEATURE COMPOSITA: CODICE GERARCHICO
# ═══════════════════════════════════════════════════════════════════════════
print("\n" + "=" * 60)
print("🏷️ CODICE GERARCHICO")
print("=" * 60)

# Creo un codice gerarchico combinando i livelli
df_multi_grain['hierarchical_code'] = (
    df_multi_grain['cluster_k2'].astype(str) + '_' +
    df_multi_grain['cluster_k4'].astype(str) + '_' +
    df_multi_grain['cluster_k8'].astype(str)
)

print("\n📋 Esempi codici gerarchici:")
print(df_multi_grain['hierarchical_code'].value_counts().head(10))

# ═══════════════════════════════════════════════════════════════════════════
# 5. COMBINO CON DATI ORIGINALI
# ═══════════════════════════════════════════════════════════════════════════
print("\n" + "=" * 60)
print("📦 DATASET FINALE")
print("=" * 60)

df_final = pd.concat([df_blobs.reset_index(drop=True), df_multi_grain], axis=1)

print(f"\n📐 Shape finale: {df_final.shape}")
print(f"\n📋 Prime 5 righe:")
print(df_final.head())

# ═══════════════════════════════════════════════════════════════════════════
# VISUALIZZAZIONE
# ═══════════════════════════════════════════════════════════════════════════
fig, axes = plt.subplots(2, 3, figsize=(15, 10))

# Row 1: Cluster a diversi livelli
for idx, k in enumerate(k_values):
    ax = axes[0, idx]
    scatter = ax.scatter(df_final['x'], df_final['y'], 
                          c=df_final[f'cluster_k{k}'], cmap='viridis', 
                          alpha=0.6, s=30)
    ax.set_xlabel('x')
    ax.set_ylabel('y')
    ax.set_title(f'🎯 Cluster k={k}')
    plt.colorbar(scatter, ax=ax, label='Cluster')

# Row 2: Distanze
for idx, k in enumerate(k_values):
    ax = axes[1, idx]
    for c in range(k):
        mask = df_final[f'cluster_k{k}'] == c
        ax.hist(df_final.loc[mask, f'dist_k{k}'], bins=15, alpha=0.5, 
                label=f'Cluster {c}')
    ax.set_xlabel('Distanza dal Centroide')
    ax.set_ylabel('Frequenza')
    ax.set_title(f'📏 Distanze k={k}')
    if k <= 4:  # Legenda solo per pochi cluster
        ax.legend(fontsize=8)

plt.tight_layout()
plt.show()

# Visualizzazione gerarchica
fig2, ax = plt.subplots(figsize=(12, 5))

# Matrice annidamento k8 → k2
combined = df_multi_grain[['cluster_k2', 'cluster_k4', 'cluster_k8']].copy()
combined['count'] = 1
pivot = combined.groupby(['cluster_k2', 'cluster_k4', 'cluster_k8']).count().reset_index()
pivot_matrix = pd.crosstab(
    [df_multi_grain['cluster_k2'], df_multi_grain['cluster_k4']], 
    df_multi_grain['cluster_k8']
)
im = ax.imshow(pivot_matrix.values, cmap='YlOrRd', aspect='auto')
ax.set_xlabel('Cluster k=8')
ax.set_ylabel('(k=2, k=4)')
ax.set_xticks(range(8))
ax.set_yticks(range(len(pivot_matrix.index)))
ax.set_yticklabels([str(x) for x in pivot_matrix.index])
ax.set_title('🔗 Annidamento Gerarchico: Come k=8 si distribuisce in (k=2, k=4)')
plt.colorbar(im, ax=ax, label='Conteggio')
plt.tight_layout()
plt.show()

print("\n" + "=" * 60)
print("✅ ESERCIZIO 27.3 COMPLETATO!")
print("=" * 60)
print("""
📝 Features Multi-Grain create:
• cluster_k2, cluster_k4, cluster_k8 → Labels a 3 livelli
• dist_k2, dist_k4, dist_k8 → Distanze per ogni livello
• hierarchical_code → Codice combinato "2_4_8"

💡 USO: Cattura pattern a diverse scale di granularità!
""")

---

# 🎯 4. Conclusione

## 🏆 Concetti Chiave Appresi

| Concetto | Descrizione |
|----------|-------------|
| **Feature Engineering Unsupervised** | Creare features senza usare labels |
| **PCA Features** | Componenti, loadings, reconstruction error |
| **Clustering Features** | Labels, distanze, probabilità soft |
| **Anomaly Features** | Scores normalizzati, flag binari |
| **Feature Selection** | Variance threshold, correlation filter |
| **Multi-Grain** | Clustering a più livelli di granularità |

## ⚠️ Errori Comuni da Evitare

| Errore | Problema | Soluzione |
|--------|----------|-----------|
| Non scalare prima di PCA | Varianze incomparabili | Sempre StandardScaler |
| Troppe features correlate | Ridondanza, overfitting | Correlation filter |
| Ignorare variance threshold | Features non informative | Rimuovere costanti |
| Dimenticare normalizzazione scores | Scores incomparabili | Normalizzare [0,1] |

## 🚀 Prossimo Passo

**Lezione 28**: Applicheremo TUTTO in un progetto end-to-end completo di Unsupervised Learning!

---

# 📚 6. Bignami — Scheda di Riferimento Rapido

## 📖 Tipologie di Features Unsupervised

| Fonte | Features | Codice |
|-------|----------|--------|
| **PCA** | Componenti | `pca.fit_transform(X)` |
| **PCA** | Rec. Error | `||X - X_rec||` |
| **K-Means** | Cluster Label | `kmeans.predict(X)` |
| **K-Means** | Distanze | `cdist(X, centroids)` |
| **GMM** | Probabilità | `gmm.predict_proba(X)` |
| **IsoForest** | Anomaly Score | `iso.decision_function(X)` |
| **LOF** | Outlier Score | `-lof.negative_outlier_factor_` |

## 🔧 Template Feature Engineering

```python
# === PIPELINE COMPLETA ===
from sklearn.preprocessing import StandardScaler
from sklearn.decomposition import PCA
from sklearn.cluster import KMeans
from sklearn.ensemble import IsolationForest

# 1. Scaling
scaler = StandardScaler()
X_scaled = scaler.fit_transform(X)

# 2. PCA Features
pca = PCA(n_components=5)
X_pca = pca.fit_transform(X_scaled)

# 3. Clustering Features
kmeans = KMeans(n_clusters=4, random_state=42)
cluster_labels = kmeans.fit_predict(X_scaled)

# 4. Anomaly Features
iso = IsolationForest(contamination=0.05, random_state=42)
anomaly_score = iso.decision_function(X_scaled)

# 5. Combine
features = pd.DataFrame({
    'pc1': X_pca[:, 0],
    'cluster': cluster_labels,
    'anomaly': anomaly_score
})
```

## 🔍 Feature Selection

```python
from sklearn.feature_selection import VarianceThreshold

# 1. Variance Threshold
selector = VarianceThreshold(threshold=0.01)
X_selected = selector.fit_transform(X_scaled)

# 2. Correlation Filter
corr = X.corr().abs()
upper = corr.where(np.triu(np.ones(corr.shape), k=1).astype(bool))
to_drop = [col for col in upper.columns if any(upper[col] > 0.9)]
X_final = X.drop(columns=to_drop)
```

## ✅ Checklist Feature Engineering

- [ ] Dati scalati con StandardScaler?
- [ ] Features a bassa varianza rimosse?
- [ ] Correlazioni elevate gestite?
- [ ] PCA componenti aggiunte?
- [ ] Cluster features aggiunte?
- [ ] Anomaly scores aggiunti?
- [ ] Features validate e testate?

---

**🎉 Fine Lezione 27 — Unsupervised Feature Engineering Masterclass!**